In [1]:
import sys
import copy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
sys.path.append("../gotmtool")
from gotmtool import *

## Create a model
Create a model with environment file `../../.gotm_env.yaml`, which is created by `gotm_env_init.py`. 

In [2]:
m = Model(name='Variable_forcing_x5', environ='../gotmtool/.gotm_env.yaml')

Take a look at what are defined in the environment file.

In [3]:
for key in m.environ:
    print('{:>15s}: {}'.format(key, m.environ[key]) )

   gotmdir_code: /Users/qingli/work/saildrone_southern_ocean/gotm/code
   gotmdir_data: /Users/qingli/work/saildrone_southern_ocean/gotm/data
  gotmdir_build: /Users/qingli/work/saildrone_southern_ocean/gotm/build
    gotmdir_exe: /Users/qingli/work/saildrone_southern_ocean/gotm/exe
    gotmdir_run: /Users/qingli/work/saildrone_southern_ocean/gotm/run
 gotmdir_figure: /Users/qingli/work/saildrone_southern_ocean/gotm/figure
   gotmdir_tool: /Users/qingli/work/saildrone_southern_ocean/gotmtool


## Build the model

In [4]:
%%time
m.build()

CPU times: user 1.07 ms, sys: 7.4 ms, total: 8.47 ms
Wall time: 115 ms


## Configuration
Initialize the GOTM configuration

In [5]:
cfg = m.init_config()

Generating default configuration at '/Users/qingli/work/saildrone_southern_ocean/gotm/run/Variable_forcing_x5/gotm.yaml'...
Done!


 ------------------------------------------------------------------------
 GOTM started on 2021/10/05 at 21:38:55
 ------------------------------------------------------------------------
    initialize_gotm
 ------------------------------------------------------------------------
        Configuration file gotm.yaml not found. Using default settings.
        configuring modules ....
    init_airsea_yaml
        done
    init_observations_yaml
    init_stokes_drift_yaml
        done
    init_turbulence_yaml
        done.
    init_cvmix_yaml
        done.
    init_meanflow_yaml
        done
    init_eqstate_yaml
        done.
 Your configuration has been written to /Users/qingli/work/saildrone_southern_ocean/gotm/run/Variable_forcing_x5/gotm.yaml.
STOP 0


Update the configuration

In [6]:
# setup
title = 'Variable forcing'
nlev = 600
cfg['title'] = title
cfg['location']['name'] = 'Idealized'
cfg['location']['latitude'] = -53.15
cfg['location']['longitude'] = -142.9
cfg['location']['depth'] = 300
cfg['time']['start'] = '2019-02-09 00:00:00'
cfg['time']['stop']  = '2019-02-11 00:00:00'
cfg['time']['dt']    = 60.0
cfg['grid']['nlev']  = nlev

# output
cfg['output'] = {}
cfg['output']['gotm_out'] = {}
cfg['output']['gotm_out']['use'] = True
cfg['output']['gotm_out']['title'] = title
cfg['output']['gotm_out']['k1_stop'] = nlev+1
cfg['output']['gotm_out']['k_stop'] = nlev
cfg['output']['gotm_out']['time_unit'] = 'dt'
cfg['output']['gotm_out']['time_step'] = 10
cfg['output']['gotm_out']['variables'] = [{}]
cfg['output']['gotm_out']['variables'][0]['source'] = '*'

# forcing
datadir = '/Users/qingli/work/saildrone_southern_ocean/case2'
cfg['temperature']['method'] = 'file'
cfg['salinity']['method'] = 'file'
cfg['surface']['fluxes']['heat']['method'] = 'file'
cfg['surface']['fluxes']['tx']['method'] = 'file'
cfg['surface']['fluxes']['tx']['column'] = 1
cfg['surface']['fluxes']['tx']['scale_factor'] = 5.0
cfg['surface']['fluxes']['ty']['method'] = 'file'
cfg['surface']['fluxes']['ty']['column'] = 2
cfg['surface']['fluxes']['ty']['scale_factor'] = 5.0
cfg['surface']['swr']['method'] = 'file'
cfg['surface']['swr']['file'] = datadir+'/swr_era5.dat'
cfg['surface']['precip']['method'] = 'constant'
cfg['waves']['stokes_drift']['us']['method'] = 'off'
cfg['waves']['stokes_drift']['vs']['method'] = 'off'

# EOS -- use linear
cfg['eq_state']['form'] = 'full-pot'

# water type (Jerlov IB)
cfg['light_extinction']['method'] = 'jerlov-ib'

In [7]:
cfg['turbulence']['turb_method'] = 'second_order'
cfg['turbulence']['tke_method'] = 'tke'
cfg['turbulence']['len_scale_method'] = 'gls'
cfg['turbulence']['scnd']['method'] =  'weak_eq_kb_eq'
cfg['turbulence']['scnd']['scnd_coeff'] =  'canuto-a'
cfg['turbulence']['turb_param']['length_lim'] = 'false'
cfg['turbulence']['turb_param']['compute_c3'] = 'true'
cfg['turbulence']['turb_param']['Ri_st'] = 0.25
cfg['turbulence']['generic']['gen_m'] = 1.5 
cfg['turbulence']['generic']['gen_n'] = -1.0
cfg['turbulence']['generic']['gen_p'] = 3.0 
cfg['turbulence']['generic']['cpsi1'] = 1.44
cfg['turbulence']['generic']['cpsi2'] = 1.92
cfg['turbulence']['generic']['cpsi3minus'] = -0.63
cfg['turbulence']['generic']['cpsi3plus'] = 1.0 
cfg['turbulence']['generic']['sig_kpsi'] = 1.0 
cfg['turbulence']['generic']['sig_psi'] = 1.3

In [8]:
# collect the configurations and the labels of the two runs
cfgs = []
labels = []

In [9]:
argo_list = ['4', '8']
sf_list = ['filtered', 'unfiltered']
for akey in argo_list:
    for fkey in sf_list:
        rlabel = '_'.join(['argo{}'.format(akey), fkey])
        print(rlabel)
        cfg['temperature']['file'] = datadir+'/t_prof{}.dat'.format(akey)
        cfg['salinity']['file'] = datadir+'/s_prof{}.dat'.format(akey)
        cfg['surface']['fluxes']['heat']['file'] = datadir+'/hf_{}.dat'.format(fkey)
        cfg['surface']['fluxes']['tx']['file'] = datadir+'/tau_{}.dat'.format(fkey) 
        cfg['surface']['fluxes']['ty']['file'] = datadir+'/tau_{}.dat'.format(fkey)
        cfg['turbulence']['turb_method'] = 'second_order' 
        cfgs.append(copy.deepcopy(cfg))
        labels.append(rlabel+'_GLS-C01A')
        cfg['turbulence']['turb_method'] = 'cvmix'
        cfg['cvmix']['surface_layer']['kpp']['langmuir_method'] = 'none'
        cfgs.append(copy.deepcopy(cfg))
        labels.append(rlabel+'_KPP-CVMix')

argo4_filtered
argo4_unfiltered
argo8_filtered
argo8_unfiltered


## Run the model

In [10]:
%%time
sims = m.run_batch(configs=cfgs, labels=labels, nproc=2)

CPU times: user 10.4 ms, sys: 13.1 ms, total: 23.5 ms
Wall time: 3.28 s
